In [2]:
%matplotlib ipympl

import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy.integrate import cumtrapz
from scipy.signal import welch

In [3]:
ds = xr.open_dataset('data/MBARI_StationM_ADCP_201711_201811.nc')
ds_alt = xr.open_dataset('data/jpl_sla_uv_timeseries.nc')

In [11]:
ds_alt

<xarray.Dataset>
Dimensions:    (time: 187)
Coordinates:
    Longitude  float32 ...
    Latitude   float32 ...
    lon        float32 ...
    lat        float32 ...
  * time       (time) datetime64[ns] 2015-11-26T12:00:00 ... 2018-06-13T12:00:00
Data variables:
    va         (time) float64 ...
    sla_err    (time) float64 ...
    ua         (time) float64 ...
    sla        (time) float64 ...

In [15]:
plt.figure()
plt.plot(ds['time'],ds['Eas_filt'][:,15]);
plt.plot(ds_alt['time'],ds_alt['ua']/10)
plt.ylabel('m/s')
plt.title('eastward velocity')

plt.figure()
plt.plot(ds['time'],ds['Nor_filt'][:,15]);
plt.plot(ds_alt['time'],ds_alt['va']/10)
plt.ylabel('m/s')
plt.title('northward velocity')

FigureCanvasNbAgg()

FigureCanvasNbAgg()

Text(0.5, 1.0, 'northward velocity')

In [8]:
t1=np.datetime64('2018-02-01')
t2=np.datetime64('2018-02-15')

plt.figure()
plt.plot(ds['time'],ds['Eas'][:,15]);
plt.plot(ds['time'],ds['Nor'][:,15]);
plt.ylabel('m/s')
plt.xlim([t1,t2])
plt.ylim([-0.15,0.15])
plt.legend(['East','North'])
plt.title('full velocity, '+str(t1)+'-'+str(t2)+', bin 15')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'full velocity, 2018-02-01-2018-02-15, bin 15')

In [9]:
t1=np.datetime64('2018-02-01')
t2=np.datetime64('2018-02-15')

plt.figure()
plt.plot(ds['time'],ds['Eas'][:,15]-ds['Eas_tide'][:,15]);
plt.plot(ds['time'],ds['Nor'][:,15]-ds['Nor_tide'][:,15]);
plt.legend(['Bin 15','Bin 1'])
plt.ylabel('m/s')
plt.ylim([-0.15,0.15])
plt.xlim([t1,t2])
plt.legend(['East','North'])
plt.title('detided velocity, '+str(t1)+'-'+str(t2)+', bin 15')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'detided velocity, 2018-02-01-2018-02-15, bin 15')

In [10]:
Eas = np.array(ds['Eas'])
Nor = np.array(ds['Nor'])
Eas_tide = np.array(ds['Eas_tide'])
Nor_tide = np.array(ds['Nor_tide'])

N = len(Eas)
fE,SE = welch(Eas[:,5],fs=288.,nperseg=N/6,window='hanning')
fN,SN = welch(Nor[:,5],fs=288.,nperseg=N/6,window='hanning')

#rotary spectrum
fr,Sr = welch(Eas[:,5]+1j*Nor[:,5],
              fs=288.,nperseg=N/6,
              window='hann',return_onesided=False)
icw, = np.where(fr<0)
iccw, = np.where(fr>0)
fcw = -fr[icw]
Scw = Sr[icw]
fccw = fr[iccw]
Sccw = Sr[iccw]

In [66]:
plt.figure()
plt.loglog(fcw,Scw)
plt.loglog(fccw,Sccw)
plt.legend(['CW','CCW'])

FigureCanvasNbAgg()

In [67]:
import pycwt as wavelet

dat = Eas[:,0]
dt = 1/288 # time step in days

dat_nomean = dat - np.mean(dat)
std = dat_nomean.std() 
var = std ** 2  
dat_norm = dat_nomean / std 

mother = wavelet.Morlet(6)
s0 = 2 * dt  # Starting scale
dj = 1 / 30  # Twelve sub-octaves per octaves
J = 9 / dj  # Seven powers of two with dj sub-octaves
alpha, _, _ = wavelet.ar1(dat)  # Lag-1 autocorrelation for red noise

wave, scales, freqs, coi, fft, fftfreqs = wavelet.cwt(dat_norm, dt, dj, s0, J,
                                                      mother)
iwave = wavelet.icwt(wave, scales, dt, dj, mother) * std

power = (np.abs(wave)) ** 2
#power /= scales[:, None]
fft_power = np.abs(fft) ** 2
period = 1 / freqs

signif, fft_theor = wavelet.significance(1.0, dt, scales, 0, alpha,
                                         significance_level=0.95,
                                         wavelet=mother)
sig95 = np.ones([1, N]) * signif[:, None]
sig95 = power / sig95

glbl_power = power.mean(axis=1)
dof = N - scales  # Correction for padding at edges
glbl_signif, tmp = wavelet.significance(var, dt, scales, 1, alpha,
                                        significance_level=0.95, dof=dof,
                                        wavelet=mother)

t = np.array(ds['time'])

# Prepare the figure
plt.figure(figsize=(8,4))

# First sub-plot, the original time series anomaly and inverse wavelet
# transform.
plt.plot(ds['time'], iwave, '-', linewidth=1, color=[0.5, 0.5, 0.5])
plt.plot(ds['time'], dat, 'k', linewidth=1.5)

# Second sub-plot, the normalized wavelet power spectrum and significance
# level contour lines and cone of influece hatched area. Note that period
# scale is logarithmic.
plt.figure(figsize=(8,5))
plt.contourf(t, period, np.log2(power), cmap=plt.cm.viridis)
plt.colorbar()
extent = [t.min(), t.max(), 0, max(period)]
plt.contour(t, period, sig95, [-99, 1], colors='k', linewidths=2,
           extent=extent)

# bx.fill(np.concatenate([t, t[-1:] + dt, t[-1:] + dt,
#                            t[:1] - dt, t[:1] - dt]),
#         np.concatenate([np.log2(coi), [1e-9], np.log2(period[-1:]),
#                            np.log2(period[-1:]), [1e-9]]),
#         'k', alpha=0.3, hatch='x')
# bx.set_title('b) {} Wavelet Power Spectrum ({})'.format(label, mother.name))
# bx.set_ylabel('Period (years)')
#
# Yticks = 2 ** np.arange(np.ceil(np.log2(period.min())),
#                            np.ceil(np.log2(period.max())))
# bx.set_yticks(np.log2(Yticks))
# bx.set_yticklabels(Yticks)

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/fftpack/basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x


FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/numpy/core/numeric.py:501: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


FigureCanvasNbAgg()

### Future processing steps
* Fill small gaps
* QC based on correlation, percent good, etc.

### Ideas for future analysis

* Wavelet analysis: isolate time variability of inertial band
* Relate inertial currents to local wind
* Complex correlation of low-pass filtered currents
* Is low frequency variability due to eddies? Relate to satellite altimeter data
* Explore relationship to local and remote wind stress curl
* Read Clarke paper on ("too fast") Rossby wave propagation in CalCOFI region
* Compare tidal currents to OSU model
* Progressive vector diagram
* Rover data - Lomb-Scargle periodogram - analysis of eddy variability at O(100 day) periods - see Stabeno et al.